In [1]:
%%javascript
IPython.notebook.kernel.execute('nb_name = "' + IPython.notebook.notebook_name + '"')

<IPython.core.display.Javascript object>

In [2]:
%load_ext autoreload
%autoreload
from IPython.display import clear_output

import os
os.environ['PYTHONHASHSEED'] = '0'
os.environ["NUMEXPR_NUM_THREADS"] = "1"
os.environ["MKL_NUM_THREADS"] = "1"
os.environ["OMP_NUM_THREADS"] = "1"

import pdb
import _pickle as cPickle
import time
import subprocess
import glob

import random
import numpy as np
import pandas as pd
import tensorflow as tf

from collections import defaultdict, Counter
from rcrp import Topic, Doc, init, sample, get_perplexity, get_topic_specialization, get_hierarchical_affinities, get_freq_tokens_rcrp, get_docs
from configure import get_config

# load config & data 

In [3]:
config = get_config(nb_name)
np.random.seed(config.seed)
random.seed(config.seed)

In [4]:
instances_train, instances_dev, instances_test, word_to_idx, idx_to_word, bow_idxs = cPickle.load(open(config.path_data,'rb'))

In [5]:
config.n_doc = len(instances_train)
config.n_vocab = len(bow_idxs)
config.n_doc, config.n_vocab

(9006, 1995)

In [6]:
docs_bow = [instance.bow for instance in instances_train]
docs_raw = [[[bow_index]*int(doc_bow[bow_index]) for bow_index in np.where(doc_bow > 0)[0]] for doc_bow in docs_bow]
docs_words = [[idx for idxs in doc for idx in idxs] for doc in docs_raw]
np.sum([len(doc_words) for doc_words in docs_words])

695746

# run

## initialize log

In [7]:
checkpoint = []
ppl_min = np.inf
epoch = 0

cmd_rm = 'rm -r %s' % config.dir_model
res = subprocess.call(cmd_rm.split())
cmd_mk = 'mkdir %s' % config.dir_model
res = subprocess.call(cmd_mk.split())

log_df = pd.DataFrame(columns=pd.MultiIndex.from_tuples(
                    list(zip(*[['','','','TRAIN:','VALID:','TEST:','SPEC:', '', '', 'HIER:', ''],
                            ['Time','Ep','Ct','PPL','PPL', 'PPL','1', '2', '3', 'CHILD', 'OTHER']]))))

def update_checkpoint(config, checkpoint, epoch):
    checkpoint.append(config.path_model + '-%i' % epoch)
    if len(checkpoint) > config.max_to_keep:
        path_model = checkpoint.pop(0)
        for p in glob.glob(path_model):
            os.remove(p)
    cPickle.dump(checkpoint, open(config.path_checkpoint, 'wb'))

## initialize data

In [8]:
topic_root = Topic(idx='0', sibling_idx=0, parent=None, depth=1, config=config)
train_docs = get_docs(instances_train, config, topic_root=topic_root)
dev_docs = get_docs(instances_dev, config, topic_root=topic_root)
test_docs = get_docs(instances_test, config, topic_root=topic_root)
init(train_docs, dev_docs, test_docs, topic_root)

## run

In [9]:
time_start = time.time()
while epoch < config.n_epochs:
    sample(train_docs, dev_docs, test_docs, topic_root)
#     ppl_train = get_perplexity(train_docs, topic_root)
    ppl_train = 0.
    ppl_dev = get_perplexity(dev_docs, topic_root)
    
    if ppl_dev < ppl_min:
        ppl_min = ppl_dev
        ppl_test = get_perplexity(test_docs, topic_root)
        cPickle.dump([test_docs, topic_root], open(config.path_model + '-%i'%epoch, 'wb'))
        update_checkpoint(config, checkpoint, epoch)
        
    depth_spec = get_topic_specialization(test_docs, topic_root)
    hierarchical_affinities = get_hierarchical_affinities(topic_root)
    
    clear_output()
    time_log = int(time.time() - time_start)
    time_start = time.time()
    log_series = pd.Series([time_log, epoch, 0, \
            '%.0f'%ppl_train, '%.0f'%ppl_dev, '%.0f'%ppl_test, \
            '%.2f'%depth_spec[1], '%.2f'%depth_spec[2] if 2 in depth_spec else 0, '%.2f'%depth_spec[3] if 3 in depth_spec else 0, \
            '%.2f'%hierarchical_affinities[0], '%.2f'%hierarchical_affinities[1]],
            index=log_df.columns)
    log_df.loc[epoch] = log_series
    display(log_df)
    get_freq_tokens_rcrp(topic_root, idx_to_word, bow_idxs)
    
    cPickle.dump(log_df, open(config.path_log, 'wb'))
    epoch += 1

TRAIN: VALID: TEST: SPEC:             HIER:      
    Time  Ep Ct    PPL    PPL   PPL     1     2     3 CHILD OTHER
0   8562   0  0      0    948   759  0.02  0.48  0.43  0.60  0.46
1   9017   1  0      0    856   683  0.02  0.51  0.44  0.51  0.44
2   7991   2  0      0    812   647  0.02  0.53  0.44  0.54  0.46
3   5797   3  0      0    785   625  0.02  0.54  0.45  0.53  0.45
4   3312   4  0      0    769   610  0.02  0.54  0.45  0.59  0.43
5   3259   5  0      0    755   600  0.02  0.54  0.45  0.53  0.43
6   3358   6  0      0    743   592  0.02  0.55  0.46  0.56  0.42
7   3313   7  0      0    733   585  0.02  0.54  0.46  0.58  0.43
8   3293   8  0      0    725   579  0.02  0.54  0.46  0.50  0.44
9   3158   9  0      0    721   575  0.02  0.55  0.47  0.52  0.40
10  3218  10  0      0    716   569  0.02  0.55  0.47  0.51  0.40
11  3324  11  0      0    710   565  0.02  0.55  0.47  0.48  0.41
12  3216  12  0      0    706   563  0.02  0.55  0.48  0.46  0.39
13  3318  13  0      0    703   559  0.02  0.55  0.48  0.49  0.40
14  3439  14  0      0    699   556  0.02  0.55  0.48  0.51  0.41

   0 : ['0-1', '0-2', '0-3', '0-4', '0-5', '0-6', '0-7', '0-8', '0-9', '0-10', '0-11', '0-12', '0-13', '0-14', '0-15', '0-16', '0-17', '0-18', '0-19', '0-20', '0-21', '0-22', '0-23', '0-24', '0-25', '0-26', '0-27', '0-28', '0-29', '0-30', '0-31', '0-32', '0-33', '0-34', '0-35', '0-37', '0-38', '0-43', '0-49', '0-50', '0-52', '0-53'] 88 2663.0 ['use', 'write', 'one', 'get', 'article', 'know', 'like', 'make', 'say', 'go']
     0-1 : ['0-1-1', '0-1-2', '0-1-3'] 8825 85062.0 ['write', 'article', 'get', 'know', 'like', 'one', 'go', 'think', 'anyone', 'make']
       0-1-1 : [] 64 1573.0 ['georgia', 'cost', 'university', 'michael', 'nasa', 'say', 'like', 'work', 'problem', 'fee']
       0-1-2 : [] 25 531.0 ['jewish', 'come', 'thanks', 'john', 'player', 'maybe', 'article', 'somebody', 'brain', 'anyone']
       0-1-3 : [] 1 1.0 ['oo', 'bhj', 'series', 'ram', 'dave', 'knowledge', 'prevent', 'james', 'throw', 'washington']
     0-2 : ['0-2-1', '0-2-2', '0-2-3'] 4772 64005.0 ['use', 'get', 'proble

KeyboardInterrupt: 